# SPY Spreadsheet

In [1]:
# other imports
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from talib.abstract import *
import itable

# project imports
import pinkfish as pf

# format price data
pd.options.display.float_format = '{:0.2f}'.format

%matplotlib inline

In [2]:
# set size of inline plots
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

Some global data

In [3]:
symbol = 'SPY'
start = datetime.datetime(1900, 1, 1)
end = datetime.datetime.now()

Fetch symbol data from internet; do not use local cache. 

In [4]:
ts = pf.fetch_timeseries(symbol, use_cache=False)

In [5]:
ts.tail()

,high,low,open,close,volume,adj_close
date,,,,,,
2020-06-08,323.41,319.63,320.22,323.20,73641200.00,323.20
2020-06-09,323.28,319.36,320.30,320.79,77479200.00,320.79
2020-06-10,322.39,318.22,321.42,319.00,95000800.00,319.00
2020-06-11,312.15,300.01,311.46,300.61,209243600.00,300.61
2020-06-12,309.08,298.60,308.24,304.21,194529100.00,304.21


Select timeseries between start and end.  Back adjust prices relative to adj_close for dividends and splits.

In [6]:
ts = pf.select_tradeperiod(ts, start, end, use_adj=True)

In [7]:
ts.head()

,high,low,open,close,volume,adj_close
date,,,,,,
1993-01-29,26.32,26.19,26.32,26.30,1003200.00,26.30
1993-02-01,26.49,26.32,26.32,26.49,480500.00,26.49
1993-02-02,26.56,26.41,26.47,26.54,201300.00,26.54
1993-02-03,26.84,26.56,26.58,26.82,529400.00,26.82
1993-02-04,26.99,26.62,26.92,26.94,531500.00,26.94


Add technical indicator: 200 day MA

In [8]:
sma200 = SMA(ts, timeperiod=200)
ts['sma200'] = sma200

In [9]:
ts.tail()

,high,low,open,close,volume,adj_close,sma200
date,,,,,,,
2020-06-08,323.41,319.63,320.22,323.20,73641200.00,323.20,298.46
2020-06-09,323.28,319.36,320.30,320.79,77479200.00,320.79,298.62
2020-06-10,322.39,318.22,321.42,319.00,95000800.00,319.00,298.82
2020-06-11,312.15,300.01,311.46,300.61,209243600.00,300.61,298.90
2020-06-12,309.08,298.60,308.24,304.21,194529100.00,304.21,299.01


Add technical indicator: ATR

In [10]:
atr = ATR(ts, timeperiod=14)
ts['atr'] = atr

In [11]:
ts.tail()

,high,low,open,close,volume,adj_close,sma200,atr
date,,,,,,,,
2020-06-08,323.41,319.63,320.22,323.20,73641200.00,323.20,298.46,6.04
2020-06-09,323.28,319.36,320.30,320.79,77479200.00,320.79,298.62,5.89
2020-06-10,322.39,318.22,321.42,319.00,95000800.00,319.00,298.82,5.76
2020-06-11,312.15,300.01,311.46,300.61,209243600.00,300.61,298.90,6.71
2020-06-12,309.08,298.60,308.24,304.21,194529100.00,304.21,299.01,6.98


Add technical indicator: 5 day high, and 5 day low

In [12]:
high5 = pd.Series(ts.high).rolling(window=5).max()
low5 = pd.Series(ts.low).rolling(window=5).min()
ts['high5'] = high5
ts['low5'] = low5

In [13]:
ts.tail()

,high,low,open,close,volume,adj_close,sma200,atr,high5,low5
date,,,,,,,,,,
2020-06-08,323.41,319.63,320.22,323.20,73641200.00,323.20,298.46,6.04,323.41,305.10
2020-06-09,323.28,319.36,320.30,320.79,77479200.00,320.79,298.62,5.89,323.41,309.08
2020-06-10,322.39,318.22,321.42,319.00,95000800.00,319.00,298.82,5.76,323.41,309.08
2020-06-11,312.15,300.01,311.46,300.61,209243600.00,300.61,298.90,6.71,323.41,300.01
2020-06-12,309.08,298.60,308.24,304.21,194529100.00,304.21,299.01,6.98,323.41,298.60


Add technical indicator: RSI, and 2-period cumulative RSI

In [14]:
rsi2 = RSI(ts, timeperiod=2)
ts['rsi2'] = rsi2

c2rsi2 = pd.Series(ts.rsi2).rolling(window=2).sum()
ts['c2rsi2'] = c2rsi2

In [15]:
ts.tail()

,high,low,open,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2
date,,,,,,,,,,,,
2020-06-08,323.41,319.63,320.22,323.20,73641200.00,323.20,298.46,6.04,323.41,305.10,97.61,193.36
2020-06-09,323.28,319.36,320.30,320.79,77479200.00,320.79,298.62,5.89,323.41,309.08,63.11,160.72
2020-06-10,322.39,318.22,321.42,319.00,95000800.00,319.00,298.82,5.76,323.41,309.08,41.38,104.50
2020-06-11,312.15,300.01,311.46,300.61,209243600.00,300.61,298.90,6.71,323.41,300.01,5.13,46.51
2020-06-12,309.08,298.60,308.24,304.21,194529100.00,304.21,299.01,6.98,323.41,298.60,29.36,34.48


Add technical indicator: Midpoint

In [16]:
mp = (ts.high + ts.low)/2
ts['mp'] = mp

In [17]:
ts.tail()

,high,low,open,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2,mp
date,,,,,,,,,,,,,
2020-06-08,323.41,319.63,320.22,323.20,73641200.00,323.20,298.46,6.04,323.41,305.10,97.61,193.36,321.52
2020-06-09,323.28,319.36,320.30,320.79,77479200.00,320.79,298.62,5.89,323.41,309.08,63.11,160.72,321.32
2020-06-10,322.39,318.22,321.42,319.00,95000800.00,319.00,298.82,5.76,323.41,309.08,41.38,104.50,320.31
2020-06-11,312.15,300.01,311.46,300.61,209243600.00,300.61,298.90,6.71,323.41,300.01,5.13,46.51,306.08
2020-06-12,309.08,298.60,308.24,304.21,194529100.00,304.21,299.01,6.98,323.41,298.60,29.36,34.48,303.84


Add technical indicator: SMA10 of midpoint

In [18]:
sma10mp = pd.Series(ts.mp).rolling(window=10).mean()
ts['sma10mp'] = sma10mp

In [19]:
ts.head(10)

,high,low,open,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2,mp,sma10mp
date,,,,,,,,,,,,,,
1993-01-29,26.32,26.19,26.32,26.30,1003200.00,26.30,nan,nan,nan,nan,nan,nan,26.25,nan
1993-02-01,26.49,26.32,26.32,26.49,480500.00,26.49,nan,nan,nan,nan,nan,nan,26.40,nan
1993-02-02,26.56,26.41,26.47,26.54,201300.00,26.54,nan,nan,nan,nan,100.00,nan,26.49,nan
1993-02-03,26.84,26.56,26.58,26.82,529400.00,26.82,nan,nan,nan,nan,100.00,200.00,26.70,nan
1993-02-04,26.99,26.62,26.92,26.94,531500.00,26.94,nan,nan,26.99,26.19,100.00,200.00,26.80,nan
1993-02-05,26.97,26.77,26.92,26.92,492100.00,26.92,nan,nan,26.99,26.32,89.34,189.34,26.87,nan
1993-02-08,27.01,26.88,26.92,26.92,596100.00,26.92,nan,nan,27.01,26.41,89.34,178.67,26.94,nan
1993-02-09,26.82,26.67,26.82,26.73,122100.00,26.73,nan,nan,27.01,26.56,16.96,106.30,26.75,nan
1993-02-10,26.79,26.65,26.73,26.77,379600.00,26.77,nan,nan,27.01,26.62,37.28,54.25,26.72,nan


Add technical indicator: Standard Deviation

In [20]:
sd = pd.Series(ts.mp).rolling(window=10).std()
ts['sd'] = sd

In [21]:
ts.tail()

,high,low,open,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2,mp,sma10mp,sd
date,,,,,,,,,,,,,,,
2020-06-08,323.41,319.63,320.22,323.20,73641200.00,323.20,298.46,6.04,323.41,305.10,97.61,193.36,321.52,308.04,7.70
2020-06-09,323.28,319.36,320.30,320.79,77479200.00,320.79,298.62,5.89,323.41,309.08,63.11,160.72,321.32,310.29,7.99
2020-06-10,322.39,318.22,321.42,319.00,95000800.00,319.00,298.82,5.76,323.41,309.08,41.38,104.50,320.31,312.30,7.70
2020-06-11,312.15,300.01,311.46,300.61,209243600.00,300.61,298.90,6.71,323.41,300.01,5.13,46.51,306.08,312.45,7.54
2020-06-12,309.08,298.60,308.24,304.21,194529100.00,304.21,299.01,6.98,323.41,298.60,29.36,34.48,303.84,312.61,7.31


In [22]:
upper = ts.sma10mp + ts.sd*2
lower = ts.sma10mp - ts.sd*2
ts['upper'] = upper
ts['lower'] = lower

In [23]:
ts.tail()

,high,low,open,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2,mp,sma10mp,sd,upper,lower
date,,,,,,,,,,,,,,,,,
2020-06-08,323.41,319.63,320.22,323.20,73641200.00,323.20,298.46,6.04,323.41,305.10,97.61,193.36,321.52,308.04,7.70,323.45,292.63
2020-06-09,323.28,319.36,320.30,320.79,77479200.00,320.79,298.62,5.89,323.41,309.08,63.11,160.72,321.32,310.29,7.99,326.27,294.31
2020-06-10,322.39,318.22,321.42,319.00,95000800.00,319.00,298.82,5.76,323.41,309.08,41.38,104.50,320.31,312.30,7.70,327.70,296.90
2020-06-11,312.15,300.01,311.46,300.61,209243600.00,300.61,298.90,6.71,323.41,300.01,5.13,46.51,306.08,312.45,7.54,327.53,297.37
2020-06-12,309.08,298.60,308.24,304.21,194529100.00,304.21,299.01,6.98,323.41,298.60,29.36,34.48,303.84,312.61,7.31,327.23,298.00


Select a smaller time from for use with itable

In [24]:
df = ts['2019-01-01':]

In [25]:
df.head()

,high,low,open,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2,mp,sma10mp,sd,upper,lower
date,,,,,,,,,,,,,,,,,
2019-01-02,245.10,239.97,240.00,244.10,126925200.00,244.10,264.90,6.04,245.29,228.07,84.93,168.27,242.53,240.61,5.13,250.86,230.36
2019-01-03,242.52,237.74,242.19,238.27,144140700.00,238.27,264.79,6.07,245.29,233.15,15.82,100.75,240.13,239.83,4.43,248.69,230.97
2019-01-04,246.95,241.16,241.57,246.25,142628800.00,246.25,264.72,6.25,246.95,237.74,73.94,89.76,244.06,239.56,4.03,247.61,231.50
2019-01-07,249.72,245.57,246.54,248.19,103139100.00,248.19,264.66,6.10,249.72,237.74,80.49,154.43,247.65,240.25,4.78,249.80,230.70
2019-01-08,251.05,247.82,250.57,250.53,102512600.00,250.53,264.65,5.90,251.05,237.74,87.84,168.33,249.44,241.31,5.54,252.39,230.22


Use itable to format the spreadsheet.  New 5 day high has blue highlight; new 5 day low has red highlight.

In [26]:
pt = itable.PrettyTable(df, tstyle=itable.TableStyle(theme='theme1'), center=True, header_row=True, rpt_header=20)

pt.update_col_header_style(format_function=lambda x: x.upper(), text_align='right')
pt.update_row_header_style(format_function=lambda x: pd.to_datetime(str(x)).strftime('%Y/%m/%d'), text_align='right')

for col in range(pt.num_cols):
    if pt.df.columns[col] == 'volume':
        pt.update_cell_style(cols=[col], format_function=lambda x: format(x, '.0f'), text_align='right')
    else:
        pt.update_cell_style(cols=[col], format_function=lambda x: format(x, '.2f'), text_align='right')

for row in range(pt.num_rows):
    if row == 0:
        continue
    if (pt.df['high5'][row] == pt.df['high'][row]) and \
       (pt.df['high5'][row] > pt.df['high'][row-1]):
        col = df.columns.get_loc('high5')    
        pt.update_cell_style(rows=[row], cols=[col], color='blue')
    if (pt.df['low5'][row] == pt.df['low'][row]) and \
       (pt.df['low5'][row] < pt.df['low'][row-1]):
        col = df.columns.get_loc('low5')
        pt.update_cell_style(rows=[row], cols=[col], color='maroon')          

In [27]:
pt

,HIGH,LOW,OPEN,CLOSE,VOLUME,ADJ_CLOSE,SMA200,ATR,HIGH5,LOW5,RSI2,C2RSI2,MP,SMA10MP,SD,UPPER,LOWER
2019/01/02,245.10,239.97,240.00,244.10,126925200,244.10,264.90,6.04,245.29,228.07,84.93,168.27,242.53,240.61,5.13,250.86,230.36
2019/01/03,242.52,237.74,242.19,238.27,144140700,238.27,264.79,6.07,245.29,233.15,15.82,100.75,240.13,239.83,4.43,248.69,230.97
2019/01/04,246.95,241.16,241.57,246.25,142628800,246.25,264.72,6.25,246.95,237.74,73.94,89.76,244.06,239.56,4.03,247.61,231.50
2019/01/07,249.72,245.57,246.54,248.19,103139100,248.19,264.66,6.10,249.72,237.74,80.49,154.43,247.65,240.25,4.78,249.80,230.70
2019/01/08,251.05,247.82,250.57,250.53,102512600,250.53,264.65,5.90,251.05,237.74,87.84,168.33,249.44,241.31,5.54,252.39,230.22
2019/01/09,252.61,249.96,251.30,251.70,95006600,251.70,264.66,5.67,252.61,237.74,91.17,179.01,251.29,243.26,5.24,253.74,232.77
2019/01/10,252.86,249.29,250.03,252.58,96823900,252.58,264.65,5.52,252.86,241.16,93.77,184.94,251.07,244.95,4.67,254.30,235.61
2019/01/11,252.71,250.78,251.41,252.68,73858100,252.68,264.66,5.26,252.86,245.57,94.15,187.91,251.74,246.36,4.35,255.06,237.66
2019/01/14,252.02,250.17,250.61,251.14,70908200,251.14,264.67,5.06,252.86,247.82,32.27,126.42,251.10,247.18,4.40,255.97,238.39
2019/01/15,254.36,251.54,251.55,254.02,85208300,254.02,264.67,4.93,254.36,249.29,80.39,112.67,252.95,248.20,4.44,257.08,239.32
